论文中FEM图示如下：  
![](FEM图示.png)

上图网络结构如下：

**输入图像：**
- Fusion Image（记作1）

**细节补充分支（FD）和颜色校正分支（FC）：**
- FD和FC的结构几乎完全一致，除了FC最后多了一个橙色和一个后续步骤。

#### 以颜色校正分支（FC）为例：

**模块一开始：**
1. 卷积层（绿色）
2. 带泄漏的ReLU激活函数（黄色）
3. 批归一化（蓝色）
4. 最大池化（玫红色）

**模块一结束**
- 重复进行一遍模块一
- 下采样

**模块二开始：**
1. 卷积层（绿色）
2. 带泄漏的ReLU激活函数（黄色）
3. 批归一化（蓝色）
4. 最大池化（玫红色）

**模块二结束**
- 重复进行一次模块二
- 下采样
- 卷积层（绿色）
- 带泄漏的ReLU激活函数（黄色）
- 批归一化（蓝色）

**阶段性处理1结束，记作2**
- 2和1相乘
- 上采样

**模块三开始：**
1. 扩展卷积层（浅绿色）
2. 带泄漏的ReLU激活函数（黄色）
3. 批归一化（蓝色）

**模块三结束**
- 再进行一次模块三
- 处理后记作3
- 3和1相乘
- 上采样

**模块四开始：**
1. 扩展卷积层（浅绿色）
2. 带泄漏的ReLU激活函数（黄色）
3. 批归一化（蓝色）

**模块四结束**
- 再进行一次模块四
- 处理后记作4
- 4和1相乘

#### 接下来的处理：

**对于颜色校正分支（FC）：**
- Tanh激活函数（浅粉色）
- Sigmoid激活函数（橙色）
- 处理后记作5
- 5和1相除，记作7
- FC处理结束

**对于细节补充分支（FD）：**
- Tanh激活函数（浅粉色）
- 处理后记作6
- 6和1相加，记作8
- FD处理结束

**最终融合：**
- 7和8相加，得到最终图像



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeatureEnhancementModule(nn.Module):
    def __init__(self, in_channels):
        super(FeatureEnhancementModule, self).__init__()

        # 卷积块+ 带泄漏的ReLU激活函数+ 批归一化+ 最大池化
        def conv_block(in_channels, out_channels, pool=True):
            layers = [
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.LeakyReLU(0.1),
                nn.BatchNorm2d(out_channels)
            ]
            if pool:
                layers.append(nn.MaxPool2d(2))
            return nn.Sequential(*layers)
        
        # 扩展卷积块+ 带泄漏的ReLU激活函数+ 批归一化
        def dilated_conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=2, dilation=2),
                nn.LeakyReLU(0.1),
                nn.BatchNorm2d(out_channels)
            )

        # 细节补充分支（FD）
        self.fd_conv1 = conv_block(in_channels, in_channels, pool=True)
        self.fd_conv2 = conv_block(in_channels, in_channels, pool=True)
        self.fd_downsample1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, stride=2)
        
        self.fd_conv3 = conv_block(in_channels, in_channels, pool=True)
        self.fd_conv4 = conv_block(in_channels, in_channels, pool=True)
        self.fd_downsample2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, stride=2)
        
        self.fd_conv5 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.fd_relu5 = nn.LeakyReLU(0.1)
        self.fd_bn5 = nn.BatchNorm2d(in_channels)

        self.fd_dilated_conv1 = dilated_conv_block(in_channels, in_channels)
        self.fd_dilated_conv2 = dilated_conv_block(in_channels, in_channels)

        self.fd_tanh = nn.Tanh()

        # 颜色校正分支（FC）
        self.fc_conv1 = conv_block(in_channels, in_channels, pool=True)
        self.fc_conv2 = conv_block(in_channels, in_channels, pool=True)
        self.fc_downsample1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, stride=2)
        
        self.fc_conv3 = conv_block(in_channels, in_channels, pool=True)
        self.fc_conv4 = conv_block(in_channels, in_channels, pool=True)
        self.fc_downsample2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, stride=2)
        
        self.fc_conv5 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.fc_relu5 = nn.LeakyReLU(0.1)
        self.fc_bn5 = nn.BatchNorm2d(in_channels)

        self.fc_dilated_conv1 = dilated_conv_block(in_channels, in_channels)
        self.fc_dilated_conv2 = dilated_conv_block(in_channels, in_channels)

        self.fc_tanh = nn.Tanh()
        self.fc_sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 细节补充分支（FD）
        fd = self.fd_conv1(x)
        fd = self.fd_conv2(fd)
        fd = self.fd_downsample1(fd)
        
        fd = self.fd_conv3(fd)
        fd = self.fd_conv4(fd)
        fd = self.fd_downsample2(fd)
        
        fd = self.fd_conv5(fd)
        fd = self.fd_relu5(fd)
        fd = self.fd_bn5(fd)
        
        fd = self.fd_dilated_conv1(fd)
        fd = self.fd_dilated_conv2(fd)
        
        fd = self.fd_tanh(fd)
        fd_out = fd + x

        # 颜色校正分支（FC）
        fc = self.fc_conv1(x)
        fc = self.fc_conv2(fc)
        fc = self.fc_downsample1(fc)
        
        fc = self.fc_conv3(fc)
        fc = self.fc_conv4(fc)
        fc = self.fc_downsample2(fc)
        
        fc = self.fc_conv5(fc)
        fc = self.fc_relu5(fc)
        fc = self.fc_bn5(fc)
        
        fc = self.fc_dilated_conv1(fc)
        fc = self.fc_dilated_conv2(fc)
        
        fc = self.fc_tanh(fc)
        fc = self.fc_sigmoid(fc)
        fc_out = fc / x

        # 最终融合
        out = fd_out + fc_out
        return out

# 测试
in_channels = 64  # 通道数
fem = FeatureEnhancementModule(in_channels)

input_image = torch.randn(1, in_channels, 256, 256)  # 示例输入张量
output_image = fem(input_image)
print(output_image.shape)

